In [2]:
! pip install nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpzkxy3p7j".


In [8]:
%%writefile generate_input.cpp

#include <iostream>
#include <fstream>
#include <random>

int main() {
    const int size = 1000000; // Size of the input file (adjust as needed)
    const std::string filename = "input.txt"; // Output filename

    // Open the output file
    std::ofstream outputFile(filename);
    if (!outputFile.is_open()) {
        std::cerr << "Error: Unable to open file for writing.\n";
        return 1;
    }

    // Generate and write random integers to the file
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<int> dist(0, 10000); // Adjust the range as needed

    for (int i = 0; i < size; ++i) {
        int num = dist(gen);
        outputFile << num << "\n";
    }

    // Close the file
    outputFile.close();

    std::cout << "Input file generated successfully: " << filename << "\n";

    return 0;
}


Writing generate_input.cpp


In [9]:
!g++ generate_input.cpp -o generate_input
!./generate_input

Input file generated successfully: input.txt


In [6]:
%%cuda
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

#define THREADS_PER_BLOCK 256

using namespace std;

// Kernel function for Radix Sort
__global__ void radixSort(int *data, int *tempData, int size, int shift) {
    extern __shared__ int sharedCount[];
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int bdim = blockDim.x;
    int idx = tid + bid * bdim;

    // Clear shared memory
    sharedCount[tid] = 0;
    __syncthreads();

    // Compute histogram in shared memory
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        atomicAdd(&sharedCount[bucket], 1);
    }
    __syncthreads();

    // Perform parallel prefix sum (scan) on shared memory
    for (int stride = 1; stride < bdim; stride *= 2) {
        int val = 0;
        if (tid >= stride) {
            val = sharedCount[tid - stride];
        }
        __syncthreads();
        sharedCount[tid] += val;
        __syncthreads();
    }

    // Copy results to tempData
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        int position = sharedCount[bucket] + atomicAdd(&tempData[bucket], 1);
        tempData[size * bucket + position] = data[idx];
    }
}


// Function to perform Radix Sort on GPU
void radixSortGPU(int *data, int size) {
    int *tempData;
    cudaMalloc(&tempData, 256 * size * sizeof(int));
    cudaMemset(tempData, 0, 256 * size * sizeof(int));

    for (int shift = 0; shift < 32; shift += 8) {
        radixSort<<<(size + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(data, tempData, size, shift);
        cudaDeviceSynchronize();

        cudaMemcpy(data, tempData, size * 256 * sizeof(int), cudaMemcpyDeviceToDevice);
        cudaMemset(tempData, 0, 256 * size * sizeof(int));
    }

    cudaFree(tempData);
}

// Function to perform Radix Sort on CPU
void radixSortCPU(int *data, int size) {
    int *output = new int[size];
    int count[256] = {0};

    for (int shift = 0; shift < 32; shift += 8) {
        for (int i = 0; i < size; i++) {
            count[(data[i] >> shift) & 0xFF]++;
        }

        for (int i = 1; i < 256; i++) {
            count[i] += count[i - 1];
        }

        for (int i = size - 1; i >= 0; i--) {
            output[--count[(data[i] >> shift) & 0xFF]] = data[i];
        }

        for (int i = 0; i < size; i++) {
            data[i] = output[i];
        }

        memset(count, 0, sizeof(count));
    }

    delete[] output;
}

int main() {
    const int size = 1000000000;
    vector<int> data(size);

    // Generate random data for sorting
    for (int i = 0; i < size; ++i) {
        data[i] = rand() % 10000;
    }

    int *gpu_data;
    cudaMalloc((void**)&gpu_data, size * sizeof(int));
    cudaMemcpy(gpu_data, data.data(), size * sizeof(int), cudaMemcpyHostToDevice);

    // Measure CPU execution time
    auto start_cpu = chrono::high_resolution_clock::now();
    radixSortCPU(data.data(), size);
    auto end_cpu = chrono::high_resolution_clock::now();
    auto duration_cpu = chrono::duration_cast<chrono::microseconds>(end_cpu - start_cpu);

    cout << "CPU Time: " << duration_cpu.count() << " microseconds" << endl;

    // Measure GPU execution time
    auto start_gpu = chrono::high_resolution_clock::now();
    radixSortGPU(gpu_data, size);
    auto end_gpu = chrono::high_resolution_clock::now();
    auto duration_gpu = chrono::duration_cast<chrono::microseconds>(end_gpu - start_gpu);

    cudaMemcpy(data.data(), gpu_data, size * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "GPU Time: " << duration_gpu.count() << " microseconds" << endl;

    cudaFree(gpu_data);

    return 0;
}


CPU Time: 46187470 microseconds
GPU Time: 1773032 microseconds



In [7]:
%%cuda
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

#define THREADS_PER_BLOCK 256

using namespace std;

// Kernel function for Radix Sort
__global__ void radixSort(int *data, int *tempData, int size, int shift) {
    extern __shared__ int sharedCount[];
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int bdim = blockDim.x;
    int idx = tid + bid * bdim;

    // Clear shared memory
    sharedCount[tid] = 0;
    __syncthreads();

    // Compute histogram in shared memory
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        atomicAdd(&sharedCount[bucket], 1);
    }
    __syncthreads();

    // Perform parallel prefix sum (scan) on shared memory
    for (int stride = 1; stride < bdim; stride *= 2) {
        int val = 0;
        if (tid >= stride) {
            val = sharedCount[tid - stride];
        }
        __syncthreads();
        sharedCount[tid] += val;
        __syncthreads();
    }

    // Copy results to tempData
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        int position = sharedCount[bucket] + atomicAdd(&tempData[bucket], 1);
        tempData[size * bucket + position] = data[idx];
    }
}

// Function to perform Radix Sort on GPU
void radixSortGPU(int *data, int size) {
    int *tempData;
    cudaMalloc(&tempData, 256 * size * sizeof(int));
    cudaMemset(tempData, 0, 256 * size * sizeof(int));

    for (int shift = 0; shift < 32; shift += 8) {
        radixSort<<<(size + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(data, tempData, size, shift);
        cudaDeviceSynchronize();

        cudaMemcpy(data, tempData, size * 256 * sizeof(int), cudaMemcpyDeviceToDevice);
        cudaMemset(tempData, 0, 256 * size * sizeof(int));
    }

    cudaFree(tempData);
}

void radixSortCPU(int *data, int size) {
    int *output = new int[size];
    int count[256] = {0};

    for (int shift = 0; shift < 32; shift += 8) {
        for (int i = 0; i < size; i++) {
            count[(data[i] >> shift) & 0xFF]++;
        }

        for (int i = 1; i < 256; i++) {
            count[i] += count[i - 1];
        }

        for (int i = size - 1; i >= 0; i--) {
            output[--count[(data[i] >> shift) & 0xFF]] = data[i];
        }

        for (int i = 0; i < size; i++) {
            data[i] = output[i];
        }

        memset(count, 0, sizeof(count));
    }

    delete[] output;
}

int main() {
    const int size = 10;
    vector<int> data(size);

    cout << "Initial Array: ";
    for (int i = 0; i < size; ++i) {
        data[i] = rand() % 10000;
        cout << data[i] << " ";
    }
    cout << endl << endl;

    int *gpu_data;
    cudaMalloc((void**)&gpu_data, size * sizeof(int));
    cudaMemcpy(gpu_data, data.data(), size * sizeof(int), cudaMemcpyHostToDevice);

    auto start_cpu = chrono::high_resolution_clock::now();
    radixSortCPU(data.data(), size);
    auto end_cpu = chrono::high_resolution_clock::now();
    auto duration_cpu = chrono::duration_cast<chrono::microseconds>(end_cpu - start_cpu);
    cout << "CPU Sorted Array: ";
    for (int i = 0; i < size; ++i) {
        cout << data[i] << " ";
    }cout << endl;
    cout << "CPU Time: " << duration_cpu.count() << " microseconds" << endl << endl;

    auto start_gpu = chrono::high_resolution_clock::now();
    radixSortGPU(gpu_data, size);
    auto end_gpu = chrono::high_resolution_clock::now();
    auto duration_gpu = chrono::duration_cast<chrono::microseconds>(end_gpu - start_gpu);

    cudaMemcpy(data.data(), gpu_data, size * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "GPU Sorted Array: ";
    for (int i = 0; i < size; ++i) {
        cout << data[i] << " ";
    }cout << endl;
    cout << "GPU Time: " << duration_gpu.count() << " microseconds" << endl;

    cudaFree(gpu_data);

    return 0;
}


Initial Array: 9383 886 2777 6915 7793 8335 5386 492 6649 1421 

CPU Sorted Array: 492 886 1421 2777 5386 6649 6915 7793 8335 9383 
CPU Time: 9 microseconds

GPU Sorted Array: 492 886 1421 2777 5386 6649 6915 7793 8335 9383 
GPU Time: 59328 microseconds



In [10]:
%%cuda
#include <iostream>
#include <fstream>
#include <vector>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

#define THREADS_PER_BLOCK 256

using namespace std;

// Kernel function for Radix Sort
__global__ void radixSort(int *data, int *tempData, int size, int shift) {
    extern __shared__ int sharedCount[];
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int bdim = blockDim.x;
    int idx = tid + bid * bdim;

    // Clear shared memory
    sharedCount[tid] = 0;
    __syncthreads();

    // Compute histogram in shared memory
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        atomicAdd(&sharedCount[bucket], 1);
    }
    __syncthreads();

    // Perform parallel prefix sum (scan) on shared memory
    for (int stride = 1; stride < bdim; stride *= 2) {
        int val = 0;
        if (tid >= stride) {
            val = sharedCount[tid - stride];
        }
        __syncthreads();
        sharedCount[tid] += val;
        __syncthreads();
    }

    // Copy results to tempData
    if (idx < size) {
        int bucket = (data[idx] >> shift) & 0xFF;
        int position = sharedCount[bucket] + atomicAdd(&tempData[bucket], 1);
        tempData[size * bucket + position] = data[idx];
    }
}

// Function to perform Radix Sort on GPU
void radixSortGPU(int *data, int size) {
    int *tempData;
    cudaMalloc(&tempData, 256 * size * sizeof(int));
    cudaMemset(tempData, 0, 256 * size * sizeof(int));

    for (int shift = 0; shift < 32; shift += 8) {
        radixSort<<<(size + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(data, tempData, size, shift);
        cudaDeviceSynchronize();

        cudaMemcpy(data, tempData, size * 256 * sizeof(int), cudaMemcpyDeviceToDevice);
        cudaMemset(tempData, 0, 256 * size * sizeof(int));
    }

    cudaFree(tempData);
}

void radixSortCPU(int *data, int size) {
    int *output = new int[size];
    int count[256] = {0};

    for (int shift = 0; shift < 32; shift += 8) {
        for (int i = 0; i < size; i++) {
            count[(data[i] >> shift) & 0xFF]++;
        }

        for (int i = 1; i < 256; i++) {
            count[i] += count[i - 1];
        }

        for (int i = size - 1; i >= 0; i--) {
            output[--count[(data[i] >> shift) & 0xFF]] = data[i];
        }

        for (int i = 0; i < size; i++) {
            data[i] = output[i];
        }

        memset(count, 0, sizeof(count));
    }

    delete[] output;
}

int main() {
    vector<int> data;
    ifstream inputFile("input.txt");
    if (!inputFile) {
        cerr << "Error: Unable to open input file." << endl;
        return 1;
    }
    int value;
    while (inputFile >> value) {
        data.push_back(value);
    }
    inputFile.close();

    int *gpu_data;
    cudaMalloc((void**)&gpu_data, data.size() * sizeof(int));
    cudaMemcpy(gpu_data, data.data(), data.size() * sizeof(int), cudaMemcpyHostToDevice);

    auto start_cpu = chrono::high_resolution_clock::now();
    radixSortCPU(data.data(), data.size());
    auto end_cpu = chrono::high_resolution_clock::now();
    auto duration_cpu = chrono::duration_cast<chrono::microseconds>(end_cpu - start_cpu);
    cout << "CPU Time: " << duration_cpu.count() << " microseconds" << endl << endl;

    ofstream cpuResultFile("cpu_result.txt");
    if (!cpuResultFile) {
        cerr << "Error: Unable to open CPU result file." << endl;
        return 1;
    }
    for (int i = 0; i < data.size(); ++i) {
        cpuResultFile << data[i] << " ";
    }
    cpuResultFile.close();

    auto start_gpu = chrono::high_resolution_clock::now();
    radixSortGPU(gpu_data, data.size());
    auto end_gpu = chrono::high_resolution_clock::now();
    auto duration_gpu = chrono::duration_cast<chrono::microseconds>(end_gpu - start_gpu);
    cout << "GPU Time: " << duration_gpu.count() << " microseconds" << endl;

    cudaMemcpy(data.data(), gpu_data, data.size() * sizeof(int), cudaMemcpyDeviceToHost);

    ofstream gpuResultFile("gpu_result.txt");
    if (!gpuResultFile) {
        cerr << "Error: Unable to open GPU result file." << endl;
        return 1;
    }
    for (int i = 0; i < data.size(); ++i) {
        gpuResultFile << data[i] << " ";
    }
    gpuResultFile.close();

    cudaFree(gpu_data);

    return 0;
}


CPU Time: 39185 microseconds

GPU Time: 1794451 microseconds

